In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [124]:
import tensorflow as tf
import numpy as np

In [3]:
from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_datasets as tfds
tfds.disable_progress_bar()

## Load training data

In [13]:
(train_data, test_data), info = tfds.load(
    'imdb_reviews/subwords8k', 
    split = (tfds.Split.TRAIN, tfds.Split.TEST), 
    with_info=True, as_supervised=True)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to /home/jovyan/tensorflow_datasets/imdb_reviews/subwords8k/0.1.0. Subsequent calls will reuse this data.


In [14]:
padded_shapes = ([None],())
train_batches = train_data.shuffle(1000).padded_batch(10, padded_shapes = padded_shapes)
test_batches = test_data.shuffle(1000).padded_batch(10, padded_shapes = padded_shapes)

In [23]:
encoder = info.features['text'].encoder

## Create model and train

In [24]:
embedding_dim=16

In [25]:
model = keras.Sequential([
  layers.Embedding(encoder.vocab_size, embedding_dim),
  layers.GlobalAveragePooling1D(),
  layers.Dense(1, activation='sigmoid')
])

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 16)          130960    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 17        
Total params: 130,977
Trainable params: 130,977
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [28]:
history = model.fit(
    train_batches,
    epochs=10,
    validation_data=test_batches, validation_steps=20)

Epoch 1/10
2500/2500 [==============================] - 22s 9ms/step - loss: 0.6382 - accuracy: 0.6924 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
2500/2500 [==============================] - 15s 6ms/step - loss: 0.4684 - accuracy: 0.8342 - val_loss: 0.4445 - val_accuracy: 0.8450
Epoch 3/10
2500/2500 [==============================] - 15s 6ms/step - loss: 0.3651 - accuracy: 0.8784 - val_loss: 0.3716 - val_accuracy: 0.8600
Epoch 4/10
2500/2500 [==============================] - 14s 6ms/step - loss: 0.3109 - accuracy: 0.8948 - val_loss: 0.3530 - val_accuracy: 0.8950
Epoch 5/10
2500/2500 [==============================] - 14s 6ms/step - loss: 0.2756 - accuracy: 0.9056 - val_loss: 0.2982 - val_accuracy: 0.8800
Epoch 6/10
2500/2500 [==============================] - 15s 6ms/step - loss: 0.2506 - accuracy: 0.9148 - val_loss: 0.3697 - val_accuracy: 0.8400
Epoch 7/10
2500/2500 [==============================] - 14s 6ms/step - loss: 0.2335 - accuracy: 0.9220 - val_loss: 0.3251 

## Get embeddings of test words

In [139]:
embeddings_layer = model.layers[0]

In [140]:
weights = embeddings_layer.get_weights()[0]

In [113]:
def get_embeddings(text):
    encoded_input = encoder.encode(text)
    embeddings = embeddings_layer(tf.constant(encoded_input))
    return embeddings.numpy()

In [125]:
result1 = get_embeddings('happy')

In [133]:
result2 = get_embeddings('dog')

In [134]:
result3 = get_embeddings('sad')

In [135]:
np.linalg.norm(result1-result2)

1.6803305

In [136]:
np.linalg.norm(result3-result2)

3.6881213

## Save embeddings

In [142]:
encoder = info.features['text'].encoder

In [143]:
import io

encoder = info.features['text'].encoder

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for num, word in enumerate(encoder.subwords):
  vec = weights[num+1] # skip 0, it's padding.
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
out_v.close()
out_m.close()